# Génération aléatoire de nombres premiers

### Code fait par : Anass Bousseaden, Baptiste Bédouret et Alexandre Bousquet

In [2]:
from random import randint
import math 

#  Test probabiliste de primalité

### 1-Test de Fermat

In [3]:
def TestPrimaliteFermat(n):
    a=randint(1,n-1)
    if (power_mod(a,n-1,n)-1)% n ==0:  # si a^n-1 est congrus à 1 mod n
        return "premier" # n est probablement premier
    return "compose" # n est composé

In [4]:
def TestPrimaliteFermatItere(n,t):
    for i in range(t):
        a=randint(1,n-1)
        if (power_mod(a,n-1,n)-1)% n ==0:
            return "premier"
    return "compose"

### 2- Test de Miller-Rabin


In [5]:
def decomp(N): #calcule s,t tels que N=2^s*t, t impair
    s=valuation(N,2)
    t=N//2^s
    return(s,t)

In [6]:
def MillerRabin(n,a):
    (s,t)=decomp(n-1)
    if(power_mod(a,t,n)-1)% n ==0: # si a^t est congrus à 1 mod n
        return "premier" 
    for i in [0..s-1]:
        if (power_mod(a,t*2^i,n)+1) % n==0: # si a^(t*2^i) est congrus à -1 mod n
            return "premier"
    return "compose"

In [7]:
def Miller_Rabin(n,k):  # répete le test MillerRabin k fois
    for i in range (k):
        a=randint(2,n-2)
        if (MillerRabin(n,a)=="compose"):
            return False  #False si n composé
    return True           #True si n premier     

# Test de primalité

### 1-Test de Lucas-Lehmer pour les nombres de Mersennes

In [7]:
def TestLucas(s):
    n=(1<<(s)) - 1 
    for i in range (2,floor(sqrt(s))):
        if (n%i == 0): # si i divise n 
            return "compose"
    u=4
    for _ in range(s-2):
        u=power_mod(u,2,n)-2
    if(u==0):
        return "premier"
    return "compose"

In [8]:
%time TestLucas(2203), TestLucas(2039)

CPU times: user 109 ms, sys: 0 ns, total: 109 ms
Wall time: 115 ms


('premier', 'compose')

### 2-Test de Pocklington-Lehmer

In [9]:
def PocklingtonBis(n,F_PrimeFactors):
    a=randint(2,n-2)
    if(power_mod(a,n-1,n)!=1):
        return "probablement compose"
    for p in F_PrimeFactors:
        R=power_mod(a,(n-1)//p,n)
        if(gcd(R,n)!=1):
            return "probablement compose"
    return "premier"

In [10]:
def pocklington(n,F_PrimeFactors):
    T=1 # if p
    for _ in range(T):
        if(PocklingtonBis(n,F_PrimeFactors)=="premier"):
            return "premier"
    return "probablement compose"

In [11]:
n=2**521-1

In [12]:
factor(n-1)

2 * 3 * 5^2 * 11 * 17 * 31 * 41 * 53 * 131 * 157 * 521 * 1613 * 2731 * 8191 * 42641 * 51481 * 61681 * 409891 * 858001 * 5746001 * 7623851 * 34110701 * 308761441 * 2400573761 * 65427463921 * 108140989558681 * 145295143558111 * 173308343918874810521923841

In [13]:
F_partialprimefactors=[2,3,5^2,11,17,31,41,53,131,157,521,1613,2731,8191,42641,51481,61681,409891,858001,5746001,7623851,34110701,34110701]

In [14]:
%time pocklington(n,F_partialprimefactors) #On a choisi F_partialprimefactors > sqrt(n)-1 

CPU times: user 31 ms, sys: 0 ns, total: 31 ms
Wall time: 49.5 ms


'premier'

# Génération de nombres premiers

### 1-Algorithme de recherche aléatoire

In [8]:
def RandomSearch(t,k,L=[3]):
    while(true):
        n=(getrandbits(k-1)<<1)+1
        for x in L:
            if(n%x==0):
                continue
        if(Miller_Rabin(n,t)):
            return n   

In [16]:
def RandomSearch_Pgcd(t,k,ProductPrimes):
    while(true):
        n=(getrandbits(k-1)<<1)+1
        if(gcd(n,ProductPrimes)!=1):
            continue
        if(Miller_Rabin(n,t)) :
            return n  

In [17]:
%time RandomSearch(4,1024,prime_range(16)) #Borne B expérimentale optimale est 16

CPU times: user 4.77 s, sys: 0 ns, total: 4.77 s
Wall time: 4.8 s


105655885358097401961841852385267313579707976142278955355466291246068495142852524352360395094900179447180193313471184694581336451156869045904686578135895647804223717133881612908052732789442189386189118316928056178927401995710398361299199791383847994624397647500379699179085834723438538255776531180557184976973

In [18]:
n=RandomSearch(4,1024,prime_range(16))
is_prime(n)

True

### 2-Algorithme de Gordon

In [9]:
def Gordon(bits,k=20,i0=1,j0=1):
    s=RandomSearch(k,bits)
    t=RandomSearch(k,bits)
    i=i0
    r=2*i*t+1
    while(Miller_Rabin(r,k)==False):
        i=i+1
        r=2*i*t+1
    p0=2*(power_mod(s,r-2,r))*s-1
    j=j0
    p=p0+2*j*r*s
    while(Miller_Rabin(p,k)==False):
        j=j+1
        p=p0+2*j*r*s
    return p

In [10]:
%time Gordon(1024)

CPU times: user 6.89 s, sys: 139 ms, total: 7.03 s
Wall time: 6.95 s


845968316668158901246722126750903739418373882620393301910971412108215615749719493791184977579769060047279628397205668757317761388374461251997459468991380629505153036626945953335495744348376238125183705275402472294481376911893384345878645137275374726181733085045932540252546280184593224751597742725702330211621992341780596774025323119754748790704699158005157285899950049052181467580804806186490159913489688429396157560268660326073473607176763797059934896657522516620054879043604162465029151315681882849889228328341107566311729589387969004349354020140532927506966046513632419519536519178229873353784365807002230476990595801

In [21]:
n2=Gordon(1024)
is_prime(n2)

True

### 3- Algorithme DSA

In [14]:
import hashlib
import hmac

In [15]:
def DSA(l):
    #Etape 1:
    L=2048+64*l
    n=(L-1)//256
    b=(L-1)%256
    
    #Etape 2:
    g=256
    s=randint(2^255,2^400)
    
    #Convertir U1 de la base 16 à la base 10
    sha2 = hashlib.sha256
    hashHexa = sha2(str(s).encode('utf-8')).hexdigest()
    h1=ZZ(hashHexa,16)
    
    U1=int (str(h1),16)
    
    #Convertir U2 de la base 16 à la base 10
    sha2 = hashlib.sha256
    hashHexa = sha2(str(power_mod(s+1,1,2**g)).encode('utf-8')).hexdigest()
    h2=ZZ(hashHexa,16)
    
    U2=int (str(h2),16)
    
    #Faire le xor
    U=U1^^U2
   
    #Faire une liste
    old_string=str(U)
    string_list=list(old_string)
    U_list = list(map(int, string_list))

    #Mettre bit poids fort/faible et changer la liste en un nombre entier
    U_list[0]=1
    U_list[-1]=1
    Z="".join(map(str, U_list))
    q=int(str(Z))
    
    while (True):
        while(Miller_Rabin(q,18)==False):
            s=randint(2^255,2^400)
            #Convertir U1 de la base 16 à la base 10
            sha2 = hashlib.sha256
            hashHexa = sha2(str(s).encode('utf-8')).hexdigest()
            h1=ZZ(hashHexa,16)
    
            U1=int (str(h1),16)
    
            #Convertir U2 de la base 16 à la base 10
            sha2 = hashlib.sha256
            hashHexa = sha2(str(power_mod(s+1,1,2**g)).encode('utf-8')).hexdigest()
            h2=ZZ(hashHexa,16)
    
            U2=int (str(h2),16)
    
            #Faire le xor
            U=U1^^U2
   
            #Faire une liste
            old_string=str(U)
            string_list=list(old_string)
            U_list = list(map(int, string_list))

            #Mettre bit poids fort/faible et changer la liste en un nombre entier
            U_list[0]=1
            U_list[-1]=1
            Z="".join(map(str, U_list))
            q=int(str(Z))
                   
    #Etape 3:
        i=0
        j=2
    #Etape 4:
        while i<4096:
            W=0
            for k in range(0,n-1):
                
                # Convertir V de la base 16 à la base 10
                sha2 = hashlib.sha256
                hashHexa = sha2(str(power_mod(s+j+k,1,2**g)).encode('utf-8')).hexdigest()
                h3=ZZ(hashHexa,16)
                
                V=int(str(h3),16)
                W+=V*2**(256*k)
             
            sha2 = hashlib.sha256
            hashHexa = sha2(str(power_mod(s+j+n,1,2**g)).encode('utf-8')).hexdigest()
            h4=ZZ(hashHexa,16)
            
            Vn=int (str(h4),16)
            W=W+(power_mod(Vn,1,2**b))*2**(256*n)
            X=W+2**(L-1)
            c=power_mod(X,1,2*q)
            p=X-(c-1)
            if p>=2**(L-1):
                if(Miller_Rabin(p,5)==True):
                    return (p,q)
            i=i+1
            j=j+n+1

In [16]:
% time DSA(1)

CPU times: user 662 ms, sys: 54 ms, total: 716 ms
Wall time: 673 ms


(489954446196911429508949597408578747654758250781368536665852545595112085236016260759025928084139760909070274771928802719962352427203255777040342144828872613365226684637120132917607138158564412983556110207361008558757943011177477903071691373513042596661747185362231591212459792350465503694718513481719139149278187714059559882814020102018544028621698109399243805596620430793338078471229957897830228746898464753904363647781384400962566419071749494241811777744410560620651587021842986205553844678138569993898703952254026142929064780535390265070215982143614574642675632160734557791250948794891462444022572222636889579694620103562517533174027,
 167412610479999512930897469062349386488969417469342249818221185770524966957464569998408708361L)

In [17]:
(n3,n4)=DSA(1)
is_prime(n3),is_prime(n4)

(True, True)